<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/02_rgb_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Prepare utils file

!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-06-22 05:33:04--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12309 (12K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]  12.02K  --.-KB/s    in 0s      

2022-06-22 05:33:04 (116 MB/s) - ‘utils.py’ saved [12309/12309]



In [2]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [3]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

from utils import ModelCreator, get_train_val_ds, ErrorAnalyzer, get_class_weight

In [16]:
BATCH_SIZE = 32
IMG_SIZE = (224, 224)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b3/feature_vector/2'

In [6]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [12]:
# Get shuffled train and validation dataset with one-hot labels

train_ds, val_ds = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE, img_size=IMG_SIZE)

Found 16580 files belonging to 16 classes.
Found 3510 files belonging to 16 classes.


In [8]:
# Get class weights which have calculated from the previous notebook and stored in github

classes, class_weight = get_class_weight()

In [ ]:
for color, weight in zip(classes, class_weight.values()):
  print(color, "    \t=>", weight)

beige     	=> 17.94
black     	=> 13.8
blue     	=> 15.47
brown     	=> 17.92
cream     	=> 56.81
crimson     	=> 76.43
gold     	=> 56.81
green     	=> 21.2
grey     	=> 12.41
navy-blue     	=> 58.77
orange     	=> 22.37
red     	=> 16.39
silver     	=> 20.17
titanium     	=> 56.81
white     	=> 2.97
yellow     	=> 20.68


In [9]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

val_ds = (
    val_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [17]:
rgb_creator = ModelCreator(HUB_URL, 'RGB-model')
rgb_model = rgb_creator.make_model(img_size=IMG_SIZE)

Model: "RGB-model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_2 (KerasLayer)  (None, 1536)              12930622  
                                                                 
 dense_2 (Dense)             (None, 16)                24592     
                                                                 
Total params: 12,955,214
Trainable params: 24,592
Non-trainable params: 12,930,622
_________________________________________________________________


In [18]:
rgb_model.layers[0].trainable = True
rgb_creator.re_compile(rgb_model)
rgb_model.summary()

Model: "RGB-model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_2 (KerasLayer)  (None, 1536)              12930622  
                                                                 
 dense_2 (Dense)             (None, 16)                24592     
                                                                 
Total params: 12,955,214
Trainable params: 12,845,998
Non-trainable params: 109,216
_________________________________________________________________


In [14]:
# Callbacks are EarlyStopping and ModelCheckpoint

callbacks = rgb_creator.get_callbacks()

In [19]:
history = rgb_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=10,
                        validation_data=val_ds,
                        class_weight=class_weight)

Epoch 1/10
519/519 [==============================] - 204s 356ms/step - loss: 13.4428 - auc: 0.8254 - accuracy: 0.7399 - val_loss: 0.6967 - val_auc: 0.8678 - val_accuracy: 0.7815
Epoch 2/10
519/519 [==============================] - 186s 357ms/step - loss: 7.1114 - auc: 0.9284 - accuracy: 0.8551 - val_loss: 0.8094 - val_auc: 0.8286 - val_accuracy: 0.7578
Epoch 3/10
519/519 [==============================] - 188s 360ms/step - loss: 5.0840 - auc: 0.9549 - accuracy: 0.8919 - val_loss: 0.7230 - val_auc: 0.8844 - val_accuracy: 0.8231
Epoch 4/10
519/519 [==============================] - 187s 358ms/step - loss: 4.2430 - auc: 0.9633 - accuracy: 0.9033 - val_loss: 0.9264 - val_auc: 0.8252 - val_accuracy: 0.7877
Epoch 5/10
519/519 [==============================] - 188s 361ms/step - loss: 2.3696 - auc: 0.9847 - accuracy: 0.9419 - val_loss: 0.6604 - val_auc: 0.8986 - val_accuracy: 0.8587
Epoch 6/10
519/519 [==============================] - 186s 356ms/step - loss: 2.1014 - auc: 0.9863 - accuracy

In [21]:
!cp ./logs/checkpoints/RGB-model.h5 ./drive/MyDrive/checkpoints/rgb-model.h5